In [ ]:
from IPython.display import HTML
from matplotlib_inline.backend_inline import set_matplotlib_formats

# Use inline backend so CSS scaling works on PNG/SVG outputs
set_matplotlib_formats('png')

HTML("""
<style>
/* =======================
   Matplotlib figure sizing
   ======================= */

/* Images (PNG/retina) */
.jp-RenderedImage img,
.jp-OutputArea-output img {
  width: 100% !important;
  height: auto !important;
  max-width: 100% !important;
  display: block;
  box-sizing: border-box;
}

/* SVG outputs */
.jp-RenderedSVG svg,
.jp-OutputArea-output svg {
  width: 100% !important;
  height: auto !important;
}

/* ipympl / canvas-backed figures (in case %matplotlib widget is used) */
.jupyter-matplotlib canvas.mpl-canvas,
.jp-OutputArea-output canvas.mpl-canvas,
.jp-OutputArea-output canvas {
  width: 100% !important;
  height: auto !important;
  max-width: 100% !important;
}

/* Reset container widths */
.jp-OutputArea-output .matplotlib-figure,
.jp-RenderedHTMLCommon .matplotlib-figure,
.jp-OutputArea-output .output_png {
  width: 100% !important;
}

/* =======================
   ipywidgets styling
   ======================= */

/* Make slider labels bold */
.widget-label {
    font-weight: bold;
}

/* Style ipywidgets buttons */
.jp-OutputArea .widget-button,
button.widget-button {
    font-weight: bold;
    background-color: #1976d2;  /* blue background */
    color: white;               /* white text */
    border: none;
    border-radius: 6px;
    padding: 6px 12px;
    font-size: 14px;
    cursor: pointer;
}

/* Button hover effect */
.jp-OutputArea .widget-button:hover,
button.widget-button:hover {
    background-color: #1259a4;  /* darker blue */
}
</style>
""")

# 👻 Spooky Signatures – Range & Azimuth Ambiguities  

As we’ve seen in earlier notebooks, **every antenna illumination pattern comes with sidelobes**. This is an inherent property of any system that works with **waves** – you can never have just one perfect, isolated beam.  

👉 The **mainlobe** is where the radar actually points — the direction where most of the transmitted wave energy goes (and where most of the energy is received). But due to **sidelobes**, some energy “leaks” to neighboring directions as well.  

What does this mean for SAR images? When our radar is pointed at a specific area on the ground, it indeed collects echoes from that area… But it also unintentionally collects echoes from **other regions** illuminated by the sidelobes. ⚡  

These **uninvited echoes** sneak into the image, creating *spooky duplicates* of real targets.  
- When the sidelobes are in the **range direction**, we get **range ambiguities**.  
- When the sidelobes are in the **azimuth direction**, we get **azimuth ambiguities**.  

---

🔦 The illustration belows shows the antenna beam pattern projected on the ground.  
- The bright **mainlobe** is where we want the radar to look.  
- The weaker **sidelobes** still illuminate other parts of the ground, but with less energy (shown with lighter colors).  

Even though sidelobes are weaker, they can still bring in **false echoes** — the ghostly signatures in our SAR imagery. 👻

In [ ]:
def build_spotlight_beam_pattern_app():
    """
    Voilá‑friendly static spotlight beam pattern demo (fixed view, 2 sidelobes/axis):
      - Satellite flies along y=0.
      - Scene center at (0, y_center, 0).
      - Mainlobe fully colored; sidelobes only along az & rg axes (2 per side), linearly fading; slight size shrink.
      - Fixed 3D view (no sliders), with a 'Redraw' button.
    """
    import numpy as np
    import matplotlib.pyplot as plt
    from matplotlib import colors as mcolors
    from mpl_toolkits.mplot3d.art3d import Poly3DCollection
    from ipywidgets import VBox, HBox, Button, Label, Output, Layout
    from IPython.display import display, clear_output

    # ---------- Tunables ----------
    SATELLITE_VELOCITY_KMS = 7.5
    ALTITUDE_KM            = 500.0
    INCIDENCE_DEG          = 30.0
    BEAM_HALF_ANGLE_DEG    = 2.0
    TRACK_WINDOW_S         = 15.0

    # Sidelobes: exactly 2 each side along azimuth & range
    MAX_STEPS_AZ           = 2
    MAX_STEPS_RG           = 2
    SPACING_SCALE_AZ       = 1.35
    SPACING_SCALE_RG       = 1.35
    SIZE_SHRINK_PER_STEP   = 0.85

    # Opacity settings
    MAIN_ALPHA             = 0.90
    SIDE_ALPHA_START       = 0.4
    SIDE_ALPHA_STEP        = 0.15
    SIDE_ALPHA_MIN         = 0.05

    # Fixed camera
    ELEVATION_VIEW         = 25
    AZIMUTH_VIEW           = 60

    out = Output()

    # ---------- Helpers ----------
    def _ellipse_poly(cx, cy, a_az, a_rg, n=240):
        th = np.linspace(0, 2*np.pi, n, endpoint=False)  # avoid duplicate vertex
        x  = cx + a_az * np.cos(th)
        y  = cy + a_rg * np.sin(th)
        z  = np.zeros_like(th)
        return list(zip(x, y, z))

    def _new_poly3d(verts, face_rgba, edge_rgba, lw=1.0, ls='-'):
        coll = Poly3DCollection([verts], zsort='min')
        coll.set_facecolor(face_rgba)
        coll.set_edgecolor(edge_rgba)
        coll.set_linewidth(lw)
        coll.set_linestyle(ls)
        return coll

    def _alpha_for_order(k: int) -> float:
        a = SIDE_ALPHA_START - (k * SIDE_ALPHA_STEP)
        return float(np.clip(a, SIDE_ALPHA_MIN, 1.0))

    def _draw():
        inc = np.radians(INCIDENCE_DEG)
        y_center = ALTITUDE_KM * np.tan(inc)          # ground-range where we look
        ground_point = (0.0, y_center, 0.0)

        # Satellite flies along y=0
        sat_x, sat_y, sat_z = 0.0, 0.0, ALTITUDE_KM

        # Short track segment (altitude) and its ground projection
        x_path = np.linspace(-SATELLITE_VELOCITY_KMS * TRACK_WINDOW_S/2,
                              SATELLITE_VELOCITY_KMS * TRACK_WINDOW_S/2, 200)
        y_path_alt = np.zeros_like(x_path)
        z_path_alt = np.full_like(x_path, ALTITUDE_KM)

        # Beam geometry
        slant_R = np.sqrt((sat_x - ground_point[0])**2 +
                          (sat_y - ground_point[1])**2 +
                          (sat_z - ground_point[2])**2)
        hw = np.tan(np.radians(BEAM_HALF_ANGLE_DEG)) * slant_R

        # Ground ellipse axes
        a_az = hw
        a_rg = hw / max(np.cos(inc), 1e-6)

        # Spacing (slightly larger than null-to-null)
        BW_AZ = 2.0 * a_az * SPACING_SCALE_AZ
        BW_RG = 2.0 * a_rg * SPACING_SCALE_RG

        # ---- Figure ----
        fig = plt.figure(figsize=(9, 6))
        ax = fig.add_subplot(111, projection='3d')
        ax.set_title("SPOTLIGHT: Ground Illumination — mainlobe + 2 sidelobes along az & rg")
        ax.set_xlabel("X (km) — azimuth")
        ax.set_ylabel("Y (km) — ground range")
        ax.set_zlabel("Z (km) — altitude")

        # Fixed view
        ax.view_init(elev=ELEVATION_VIEW, azim=AZIMUTH_VIEW)

        # Limits — include y=0 track and lobes around y_center
        y_span_lobes = (MAX_STEPS_RG + 1) * BW_RG + 10
        y_min = min(0.0, y_center - y_span_lobes) - 10
        y_max = max(y_center + y_span_lobes, 0.0 + 10)
        x_span = max(abs(x_path.min()), abs(x_path.max()), (MAX_STEPS_AZ + 1) * BW_AZ) + 10
        ax.set_xlim(-x_span, x_span)
        ax.set_ylim(y_min, y_max)
        ax.set_zlim(0.0, ALTITUDE_KM * 1.15)

        # Context: track at altitude & ground projection (both y=0)
        ax.plot(x_path, y_path_alt, z_path_alt, lw=1.4, color='dimgray', alpha=0.9, label="Flight Path (altitude)")
        ax.plot(x_path, y_path_alt, np.zeros_like(x_path), lw=1.0, ls=':', color='0.5', label="Flight Path (ground proj.)")

        # Scene center, LOS, satellite
        ax.plot([ground_point[0]], [ground_point[1]], [0.0],
                marker='o', markersize=6, color='k', label="Scene Center")
        ax.plot([sat_x, ground_point[0]], [sat_y, ground_point[1]], [sat_z, 0.0],
                ls='--', lw=1.5, color='red', label="Beam Center (LOS)")
        ax.scatter([sat_x], [sat_y], [sat_z],
                   marker='^', s=120, color='orange', edgecolors='k', linewidths=0.8, label='Satellite')

        gold_rgb  = mcolors.to_rgb('gold')
        edge_rgba = (0, 0, 0, 1.0)

        # Mainlobe (fully colored)
        main_poly = _ellipse_poly(0.0, y_center, a_az, a_rg)
        ax.add_collection3d(_new_poly3d(main_poly, (*gold_rgb, MAIN_ALPHA), edge_rgba, lw=1.0))

        # Azimuth sidelobes: centers (±k*BW_AZ, y_center), k=1..2
        for k in range(1, MAX_STEPS_AZ + 1):
            shrink = (SIZE_SHRINK_PER_STEP ** k)
            a_az_k, a_rg_k = a_az * shrink, a_rg * shrink
            alpha = _alpha_for_order(k)
            for cx in (k * BW_AZ, -k * BW_AZ):
                poly = _ellipse_poly(cx, y_center, a_az_k, a_rg_k)
                ax.add_collection3d(_new_poly3d(poly, (*gold_rgb, alpha), (0.3, 0.3, 0.3, 1.0), lw=0.6))

        # Range sidelobes: centers (0, y_center ± k*BW_RG), k=1..2
        for k in range(1, MAX_STEPS_RG + 1):
            shrink = (SIZE_SHRINK_PER_STEP ** k)
            a_az_k, a_rg_k = a_az * shrink, a_rg * shrink
            alpha = _alpha_for_order(k)
            for cy in (y_center + k * BW_RG, y_center - k * BW_RG):
                poly = _ellipse_poly(0.0, cy, a_az_k, a_rg_k)
                ax.add_collection3d(_new_poly3d(poly, (*gold_rgb, alpha), (0.3, 0.3, 0.3, 1.0), lw=0.6))

        # Optional: dashed outline around mainlobe null‑to‑null
        null_poly = _ellipse_poly(0.0, y_center, a_az, a_rg)
        ax.add_collection3d(_new_poly3d(null_poly, (0, 0, 0, 0), (0, 0, 0, 1.0), lw=1.2, ls='--'))

        ax.legend(loc='upper left', bbox_to_anchor=(1.05, 1), borderaxespad=0., fontsize=9)
        fig.tight_layout()

        with out:
            clear_output(wait=True)
            display(fig)
            plt.close(fig)

    # ---------- UI ----------
    header = Label("SPOTLIGHT: Mid‑aperture — mainlobe + 2 sidelobes along azimuth & range (fixed view)")
    redraw_btn = Button(description="Redraw", layout=Layout(width='100px'))
    redraw_btn.on_click(lambda _: _draw())

    # Initial render
    _draw()

    return VBox([header, HBox([redraw_btn]), out])

# Build & display (Voilá‑friendly)
app = build_spotlight_beam_pattern_app()
display(app)


## 👻 Range Ambiguities  

As we've learned, in SAR we send out little **pulses of energy** 📡✨ one after another. After each pulse, the radar **listens** 👂 for a short while to catch the echoes coming back. The delay of the echo tells us how far away the target is 🗺️.  

But… there’s a catch ⚠️. The radar cannot listen forever – it must stop listening and send the **next pulse** 🔁. That listening time sets the “maximum range” the radar can measure without confusion.  

---

### 🚗 A simple story  

- Imagine a **near target** 🟥 close to the radar.  
- And a **far target** 🟦 that is much further away.  

When the radar sends a pulse ➡️, the **near target** sends its echo back quickly 🏃‍♂️💨. The radar records it nicely ✅.  

The **far target’s** echo, however, takes much longer to come back 🐢. By the time it arrives, the radar has already stopped listening for the first pulse echoes and sent the **next pulse** 🚀. So when the radar starts listening for the echoes of the new pulse, the late echo of the previous pulse from the far target sneaks in 👻.  

Because every pulse looks exactly the same 🔂, the radar has **no way of knowing** that this echo actually corresponded to the *previous* pulse that was sent out. Instead, it gets placed at the **wrong distance**, appearing as a **ghost target**.  

### 📡 The role of the antenna pattern  

Earlier we saw that every antenna has a **mainlobe** 🔦 (where we want to look) and weaker **sidelobes** 🌙 (uninvited directions).  

When it comes to **range ambiguities**, this matters a lot:  
- The radar’s **pulse rate (PRF)** is chosen so that everything inside the **mainlobe** returns safely within the listening window ✅.  
- But if a strong reflector happens to sit in a **sidelobe**, its echo may take longer (or less time) to return ⏳.  
- By the time it arrives, the radar has already moved on to the next pulse ➡️… so the echo “folds” into the wrong range and shows up as a **ghost** 👻.  

So in practice:  
- **Mainlobe targets** → safe, unambiguous echoes.  
- **Sidelobe targets** → risky, because their late (or early) echoes can masquerade as nearby objects and create **spooky duplicates** in the image.  

---

### 🔍 What you see in the next simulation  

- **Left panel – Propagation view**  
  - The white dot ⚪ is the radar, moving along track, transmitting and receiving pulse echoes as it travels along.  
  - The dashed circle ⚪⚪ shows the current **unambiguous range** (R_unamb). Its size depends on the **PRF**:  
    - Higher PRF → shorter PRI → **smaller R_unamb** (dashed circle shrinks).  
    - Lower PRF → longer PRI → **larger R_unamb** (circle grows).  
  - The red ❌ inside the circle is the **near target** → its echo returns within the listening window and is placed correctly.  
  - The red ❌ outside the circle is the **far target** → its echo arrives too late and slips into the listening time of the next pulse, creating a **ghost** 👻.  

- **Right panel – Received echoes (RTI)**  
  - Each row 📏 is one pulse repetition interval (PRI).  
  - The horizontal axis always spans from **0 to the current unambiguous range**.  
  - The near target ✅ appears at the correct range bin, in the same row as its pulse.  
  - The far target’s echo ❌ cannot fit in its original PRI, so it folds into the **next row**, showing up as if it were much closer → a **ghost echo** 👻.  

- **Interactive control – PRF slider**  
- Use the **PRF (kHz)** slider to adjust the pulse repetition frequency.  
  - Lowering the PRF increases the unambiguous range, so **both targets fit cleanly inside the window**.  
  - Raising the PRF shrinks the range window, causing the far target to fold into the next pulse as a **ghost** 👻.  

In [ ]:
def build_range_ambiguity_sim():
    """
    Range-ambiguity demo with incremental RTI (row appended exactly when a PRI completes):
      LEFT  = propagation view (with faint R_unamb circle).
      RIGHT = per-pulse echoes; one new row is appended when we ENTER a new PRI.

    Faster plotting:
      - Render both panels + colorbar server-side to a PNG and show via ipywidgets.Image
      - Keep RTI incremental commit logic and all controls unchanged
    """
    import io
    import numpy as np
    import matplotlib.pyplot as plt
    from ipywidgets import (
        FloatSlider, IntSlider, Play, jslink,
        ToggleButton, Button, HBox, VBox, Layout, Label, Image as WImage
    )

    # -----------------------
    # Constants & scene
    # -----------------------
    c = 299_792_458.0  # m/s

    # Antenna pattern (sinc^2 with a visual mainlobe width)
    beamwidth_degrees = 45.0
    def antenna_pattern(theta_rad):
        bw = np.radians(beamwidth_degrees)
        return np.sinc(theta_rad / bw) ** 2

    # Pulse timing (PRF will be user-controlled; start at 750 kHz)
    PRF     = 0.75e6         # Hz (mutable via slider)
    PRI     = 1.0 / PRF      # s
    pulse_s = 1.0e-7         # 0.1 µs pulse width (visible ring thickness)
    R_unamb = c * PRI / 2.0  # unambiguous range

    # Geometry / canvas
    space_extent = 200.0
    grid_size    = 220
    x = np.linspace(-space_extent, space_extent, grid_size)
    y = np.linspace(0.0, 2.0 * space_extent, grid_size)
    dx = x[1] - x[0]
    dy = y[1] - y[0]
    extent_img = (x[0] - dx/2, x[-1] + dx/2,  # left, right
                  y[-1] + dy/2, y[0] - dy/2)  # bottom, top (origin='upper')
    X, Y = np.meshgrid(x, y)

    # Radar motion: one new position per PRI
    num_positions = 30
    radar_x_positions = np.linspace(-0.6*space_extent, 0.6*space_extent, num_positions)
    radar_y = 0.0

    # Fixed boresight pointing to scene center
    bore_x, bore_y = 0.0, space_extent

    # Animation granularity inside each PRI
    wave_frames  = 20
    total_frames = wave_frames * num_positions

    # Defaults: T1 < R_unamb (same-PRI), T2 > R_unamb (next-PRI)
    t1x0, t1y0 =  20.0, 150.0
    t2x0, t2y0 = -20.0, 280.0

    # Precompute circle unit points (for R_unamb overlay)
    th = np.linspace(0, 2*np.pi, 360)
    cos_th, sin_th = np.cos(th), np.sin(th)

    # -----------------------
    # Widgets
    # -----------------------
    t1x = FloatSlider(description="T1 x (m)", min=-space_extent, max=space_extent, step=1.0,
                      value=t1x0, continuous_update=False, readout_format=".1f",
                      layout=Layout(width="48%"))
    t1y = FloatSlider(description="T1 y (m)", min=0.0, max=2.0*space_extent, step=1.0,
                      value=t1y0, continuous_update=False, readout_format=".1f",
                      layout=Layout(width="48%"))

    t2x = FloatSlider(description="T2 x (m)", min=-space_extent, max=space_extent, step=1.0,
                      value=t2x0, continuous_update=False, readout_format=".1f",
                      layout=Layout(width="48%"))
    t2y = FloatSlider(description="T2 y (m)", min=0.0, max=2.0*space_extent, step=1.0,
                      value=t2y0, continuous_update=False, readout_format=".1f",
                      layout=Layout(width="48%"))

    # PRF slider in kHz
    prf_slider = FloatSlider(description="PRF (kHz)", min=50.0, max=2000.0, step=10.0,
                             value=750.0, continuous_update=False, readout_format=".0f",
                             layout=Layout(width="48%"))

    play = Play(value=0, min=0, max=total_frames - 1, step=1, interval=100)
    t_slider = IntSlider(value=0, min=0, max=total_frames - 1, step=1, description="Time")
    jslink((play, 'value'), (t_slider, 'value'))

    auto_update = ToggleButton(value=True, description="Auto-update", icon="bolt")
    render_btn  = Button(description="Render", button_style="primary", icon="refresh")

    readout = Label()
    controls1 = HBox([t1x, t1y])
    controls2 = HBox([t2x, t2y])
    controls_prf = HBox([prf_slider], layout=Layout(justify_content='flex-start'))
    controls3 = HBox([play, t_slider, auto_update, render_btn],
                     layout=Layout(align_items='center', column_gap='10px'))

    # -----------------------
    # Persistent RTI state
    # -----------------------
    t_fast = None
    T_win  = None
    echo_img = None
    last_committed_row = -1  # no rows yet
    prev_pos_idx = -1        # track previous radar position index

    # PNG image widget for both panels
    out_img = WImage(format='png', layout=Layout(width="960px", height="480px"))

    # -----------------------
    # Helpers
    # -----------------------
    pulse_width_m = c * pulse_s  # ring thickness

    def add_ring(Z, cx, cy, radius, width_m, amp=1.0):
        if radius <= 0:
            return
        max_r = 2.5 * space_extent
        if radius - width_m > max_r:
            return
        d = np.hypot(X - cx, Y - cy)
        ring_mask = (np.abs(d - radius) <= (width_m / 2.0))
        if ring_mask.any():
            Z[ring_mask] = np.maximum(Z[ring_mask], amp)

    def compute_T1_track_times(T1):
        sx = radar_x_positions
        R1p = np.hypot(sx - T1[0], radar_y - T1[1])
        return 2.0 * R1p / c  # vectorized

    def build_receive_window(T1):
        """
        Window = exactly one PRI so the RTI range axis spans the *full* unambiguous range.
        """
        nonlocal PRI
        _T_win = PRI  # full PRI → axis = [0, R_unamb]
        # Keep a reasonable sampling density regardless of PRI
        fs = max(10.0 / pulse_s, 1024.0 / _T_win)
        n_bins = int(max(256, np.round(_T_win * fs)))
        _t_fast = np.linspace(0.0, _T_win, n_bins)
        return _T_win, _t_fast

    def reset_rti(T1):
        nonlocal T_win, t_fast, echo_img, last_committed_row, prev_pos_idx
        T_win, t_fast = build_receive_window(T1)
        echo_img = np.zeros((num_positions, t_fast.size), dtype=float)
        last_committed_row = -1
        prev_pos_idx = -1

    def gaussian_pulse(x, center, width):
        return np.exp(-0.5 * ((x - center) / width) ** 2)

    def commit_row(r, T1, T2):
        """
        Append (fill) exactly one row r of the RTI image.
        Row r collects echoes from all emissions p ≤ r whose arrivals fall in this PRI:
          r == p + floor((2R_p/c)/PRI)
        Plotted at folded fast time t_mod = (2R_p/c) % PRI, if within [0, T_win].

        Echo magnitudes are scaled by the transmit antenna gain toward each target,
        using a fixed boresight pointing to (0, space_extent).
        """
        if r < 0 or r >= num_positions:
            return
        width = pulse_s / 2.0

        # vectorized over emissions p = 0..r
        p_idx = np.arange(0, r + 1)
        sx = radar_x_positions[p_idx]

        # Fixed boresight azimuth for each emission position
        az_bore = np.arctan2(bore_x - sx, bore_y - radar_y)

        # T1
        R1 = np.hypot(sx - T1[0], radar_y - T1[1])
        t1 = 2.0 * R1 / c
        k1 = np.floor(t1 / PRI).astype(int)
        az_t1 = np.arctan2(T1[0] - sx, T1[1] - radar_y)
        tx_gain_1 = antenna_pattern(az_t1 - az_bore)
        mask1 = (p_idx + k1) == r
        if mask1.any():
            t1f = np.mod(t1[mask1], PRI)
            gains1 = tx_gain_1[mask1]
            for tf, g in zip(t1f, gains1):
                if 0.0 <= tf <= T_win:
                    echo_img[r, :] += 0.9 * float(g) * gaussian_pulse(t_fast, tf, width)

        # T2
        R2 = np.hypot(sx - T2[0], radar_y - T2[1])
        t2 = 2.0 * R2 / c
        k2 = np.floor(t2 / PRI).astype(int)
        az_t2 = np.arctan2(T2[0] - sx, T2[1] - radar_y)
        tx_gain_2 = antenna_pattern(az_t2 - az_bore)
        mask2 = (p_idx + k2) == r
        if mask2.any():
            t2f = np.mod(t2[mask2], PRI)
            gains2 = tx_gain_2[mask2]
            for tf, g in zip(t2f, gains2):
                if 0.0 <= tf <= T_win:
                    echo_img[r, :] += 0.7 * float(g) * gaussian_pulse(t_fast, tf, width)

    # Initialize RTI buffers with defaults
    reset_rti((t1x0, t1y0))

    # -----------------------
    # PNG rendering (server-side)
    # -----------------------
    DPI = 110
    def render_png(Z_prop, rx_x, rx_y, T1, T2, rows_to_show):
        fig, (ax_prop, ax_rti) = plt.subplots(
            1, 2, figsize=(12.8, 6.2), dpi=DPI, gridspec_kw={'width_ratios':[1.05, 0.95]}
        )

        # LEFT: propagation view (no white borders)
        ax_prop.imshow(
            Z_prop,
            extent=extent_img,
            cmap='jet',
            vmin=0.0, vmax=1.0,
            origin='upper',
            interpolation='nearest'
        )

        # lock the logical domain exactly
        ax_prop.set_xlim(x[0], x[-1])
        ax_prop.set_ylim(y[-1], y[0])  # origin='upper' → decreasing y

        # overlays
        ax_prop.scatter([T1[0], T2[0]], [T1[1], T2[1]], c="red", s=100, marker="x", label="Targets")
        ax_prop.scatter([rx_x], [rx_y], c="white", s=40, marker="o", label="Radar")
        ax_prop.plot(rx_x + R_unamb*cos_th, rx_y + R_unamb*sin_th,
                     linestyle='--', linewidth=1.0, alpha=0.5, color='white', label="Max range")

        ax_prop.set_title("Propagation (Max. range dashed)")
        ax_prop.set_xlabel("X (m)")
        ax_prop.set_ylabel("Y (m)")
        ax_prop.legend(loc='upper right')
        ax_prop.set_aspect('equal', adjustable='box')

        # RIGHT: per‑pulse echoes (incremental) — x-axis = full unambiguous range
        extent_r = [0.0, R_unamb, num_positions, 0]  # 0 → R_unamb (meters)
        show_img = np.zeros_like(echo_img)
        if rows_to_show > 0:
            show_img[:rows_to_show, :] = echo_img[:rows_to_show, :]
        im1 = ax_rti.imshow(show_img, aspect='auto', cmap='jet', extent=extent_r,
                            origin='upper', vmin=0.0, vmax=1.0)
        ax_rti.set_title("Per‑pulse echoes (fast time within one PRI)")
        ax_rti.set_xlabel("Range within PRI (m)")
        ax_rti.set_ylabel("Pulse index")

        # Single colorbar synced to RTI
        fig.colorbar(im1, ax=ax_rti, label='Echo magnitude')

        fig.tight_layout()
        buf = io.BytesIO()
        fig.savefig(buf, format='png', dpi=DPI)
        plt.close(fig)
        buf.seek(0)
        return buf.getvalue()

    # -----------------------
    # Draw routine
    # -----------------------
    def draw(frame_idx):
        nonlocal last_committed_row, prev_pos_idx

        # Time bookkeeping with current PRI
        pos_idx      = frame_idx // wave_frames
        pos_idx      = int(np.clip(pos_idx, 0, num_positions - 1))
        time_idx     = frame_idx % wave_frames
        phase_in_PRI = time_idx / max(1, (wave_frames - 1))
        t_now        = pos_idx * PRI + phase_in_PRI * PRI

        # Targets (current sliders)
        T1 = (float(t1x.value), float(t1y.value))
        T2 = (float(t2x.value), float(t2y.value))

        # LEFT panel image (compose rings)
        rx_x = radar_x_positions[pos_idx]; rx_y = radar_y
        Z = np.zeros_like(X)

        # Outgoing pulse rings from recent emissions (weighted by antenna pattern to fixed boresight)
        recent_K = 2
        k_min = max(0, pos_idx - recent_K)
        for k in range(k_min, pos_idx + 1):
            t_emit = k * PRI
            sx = radar_x_positions[k]
            r_out = c * (t_now - t_emit)

            # Per-pixel azimuth grid (from this emission position) and fixed boresight
            theta_grid = np.arctan2(X - sx, Y - rx_y)
            az_bore    = np.arctan2(bore_x - sx, bore_y - rx_y)
            pat_grid   = antenna_pattern(theta_grid - az_bore)

            # Weighted outgoing ring mask
            if r_out > 0:
                d = np.hypot(X - sx, Y - rx_y)
                ring_mask = (np.abs(d - r_out) <= (pulse_width_m / 2.0))
                if ring_mask.any():
                    Z[ring_mask] = np.maximum(Z[ring_mask], 1.0 * pat_grid[ring_mask])

            # Scatter rings from targets (include transmit gain toward the target)
            for (tx, ty, base_amp) in ((T1[0], T1[1], 0.8),
                                       (T2[0], T2[1], 0.8)):
                R_rt = np.hypot(sx - tx, rx_y - ty)
                tau  = (t_now - t_emit) - (R_rt / c)
                if tau > 0.0:
                    r_back = c * tau
                    az_t = np.arctan2(tx - sx, ty - rx_y)
                    tx_gain = float(antenna_pattern(az_t - az_bore))
                    add_ring(Z, tx, ty, r_back, pulse_width_m, amp=base_amp * tx_gain)

        # --- Incremental RTI commit: commit when we ENTER a new PRI (pos change) ---
        if pos_idx != prev_pos_idx:
            if prev_pos_idx >= 0:
                for r in range(last_committed_row + 1, prev_pos_idx + 1):
                    commit_row(r, T1, T2)
                    last_committed_row = r
            prev_pos_idx = pos_idx

        rows_to_show = max(0, last_committed_row + 1)

        # Readout (for current position)
        R1 = np.hypot(rx_x - T1[0], rx_y - T1[1])
        R2 = np.hypot(rx_x - T2[0], rx_y - T2[1])
        readout.value = (
            f"PRF={PRF/1e3:.0f} kHz, PRI={(1.0/PRF)*1e6:.2f} µs | "
            f"pos={pos_idx+1}/{num_positions}, frame={frame_idx+1}/{total_frames} | "
            f"R_unamb={R_unamb:.1f} m | T1: R={R1:.1f} m | T2: R={R2:.1f} m"
        )

        # ---- Single PNG render for both panels ----
        out_img.value = render_png(Z, rx_x, rx_y, T1, T2, rows_to_show)

    # -----------------------
    # Wiring
    # -----------------------
    def maybe_draw(change=None):
        if auto_update.value:
            draw(t_slider.value)

    def on_target_change(_):
        reset_rti((float(t1x.value), float(t1y.value)))
        maybe_draw(None)

    def on_prf_change(change):
        nonlocal PRF, PRI, R_unamb
        PRF = float(prf_slider.value) * 1e3  # kHz → Hz
        PRI = 1.0 / PRF
        R_unamb = c * PRI / 2.0
        # Rebuild RTI buffers (window = PRI so x-axis spans 0..R_unamb)
        reset_rti((float(t1x.value), float(t1y.value)))
        maybe_draw(None)

    t_slider.observe(lambda ch: draw(ch["new"]), names='value')
    for w in (t1x, t1y, t2x, t2y):
        w.observe(on_target_change, names='value')
    prf_slider.observe(on_prf_change, names='value')
    render_btn.on_click(lambda _: draw(t_slider.value))

    # Initial render
    draw(t_slider.value)

    return VBox(
        [controls_prf, controls1, controls2, controls3, readout, out_img],
        layout=Layout(row_gap='8px')
    )

# Build and show:
range_ambiguity_app = build_range_ambiguity_sim()
display(range_ambiguity_app)

## Azimuth Ambiguities  

So far, we’ve seen how **range ambiguities** sneak into our data: echoes from previous or later pulses slipping in through the **range sidelobes**.  

But sidelobes don’t only exist in range — our **real antenna pattern** also has sidelobes in the **azimuth (along-track)** direction. ⚡  
These can let in energy from outside the main look direction.  

Now, here’s where it gets interesting: When we form a SAR image, we don’t deliberately “focus” the beam of the virtual array toward those sidelobe directions. So why do they still matter?  

👉 The culprit is **sampling and aliasing**. Master Nyquist is due back for a visit!  

---

## 🌊 A Familiar Analogy: Buoy Arrays & Grating Lobes  

Remember when we played with **phased arrays**? If we placed antenna elements more than **half a wavelength apart**, we got those nasty **grating lobes** — extra beams that looked just as strong as the main one, but pointed in the wrong direction. 🚨  

Why did that happen? Because the **phase differences** between widely spaced elements became **ambiguous** for signals arriving from certain angles. In other words: the array was sampled too coarsely, so multiple directions produced the *same* phase pattern. That’s **aliasing** in action.  

Now here’s the SAR connection: **SAR is really just a gigantic *virtual* phased array!**  Instead of many physical antennas side by side, the satellite collects echoes at many points along its orbit. Each radar pulse adds a new “virtual element” to this synthetic array. 💡  

👉 If we could sample this virtual array finely enough — i.e. with “half-wavelength spacing” between virtual elements — all would be fine: no ghosts, no confusion, just a crisp image.  

But in practice, we can’t. (You’ll soon see why. 😉) As a result, the virtual array inevitably has a pattern of **grating lobes**. That means echoes from outside the scene can line up with ambiguous phase differences, and the SAR processor **misinterprets them** as coming from inside the imaged swath.  

➡️ The outcome: **ghost targets** 👻 — the SAR equivalent of array grating lobes.  

---

## 🛰️ Virtual Antenna Elements in SAR

So, what controls the **spacing** of these virtual antenna elements?  Not hardware you can move around, but something very simple: the **Pulse Repetition Frequency (PRF)**.

- The satellite moves forward at a fixed speed (we can’t change orbital velocity easily 🌍).  
- Every pulse we send out corresponds to one virtual antenna element.  
- The higher the PRF, the **closer together** those elements are along the flight path.  
- The lower the PRF, the **further apart** they are.

Sounds good — so why don’t we just crank up the PRF until our spacing is nice and tight? 🎚️

---

## ⚖️ Why PRF Can’t Save Us

Several things limit how high we can set the PRF:

- 🔧 **Hardware limitations** — the radar electronics can only handle so much.  
- 🔍 **Range ambiguities** — as we just saw, higher PRF means shorter unambiguous range, so we’d lose coverage.  
- 📊 **SAR mode trade-offs** — wide swath? fine resolution? long range? You can’t maximize everything at once.  

The essential fact remains: in spaceborne SAR, **we simply can’t set the PRF high enough** to reach the ideal “half-wavelength spacing.” 😕

And when elements are spaced too far apart? 👉 Just like with the buoy array, **grating lobes appear**. These lobes are strong, unwanted directions where energy from outside the main beam leaks into our image.

---

## 👻 The Ghost Problem  

Those azimuth grating lobes don’t just look ugly in theory — they cause real trouble in SAR images. Echoes from targets **outside the intended imaging swath** can still “sneak in” through the grating lobes of our virtual array.  

When the processor tries to focus the data, it doesn’t know these echoes came from the wrong direction. ➡️ They get folded back into the main image area, appearing as **ghost-like replicas** of bright targets, shifted along azimuth.  

These are the infamous **azimuth ambiguities**. 👻 They clutter the image, confuse interpretation, and can even bury weaker targets under phantom copies of stronger ones.  

---

🕹️ The following interactive demo lets you *see* how it happens:

- **Left panel — “SAR Data Collection”**  
  Watch pulses go out, scatter off two targets, and return.  
  - The **red target ❌** sits in the main beam.  
  - The **cyan target ❌** sits outside the main beam, in the sidelobe region.  
  Even though it’s “out of bounds,” notice how it still contributes energy.

- **Right panel — “Accumulated SAR Echoes”**  
  As the radar moves along its path, each position adds a new **range profile**. Together, these lines form the raw data used for SAR imaging — and yes, the sidelobe target sneaks into them too.


In [ ]:
def build_azimuth_ambiguities_demo():
    import numpy as np
    import matplotlib.pyplot as plt
    from ipywidgets import Play, IntSlider, jslink, HBox, VBox, Layout, Output, Label
    from IPython.display import display

    # -----------------------
    # Constants / Parameters
    # -----------------------
    c = 299_792_458.0                   # m/s
    beamwidth_degrees = 15.0            # antenna main-beam width
    pulse_duration = 5e-8               # s
    space_extent = 200.0                # m (scene half-width in x; y is [0, 2*extent])
    grid_size = 200                     # grid per dimension

    wave_frames   = 15                  # subframes per radar position
    num_positions = 15                  # aperture steps
    total_frames  = wave_frames * num_positions

    # time window long enough for pulse out + back from far edge
    pulse_time = 4.0 * space_extent / c

    # -----------------------
    # Grid (static)
    # -----------------------
    x = np.linspace(-space_extent, space_extent, grid_size)
    y = np.linspace(0, 2 * space_extent, grid_size)
    X, Y = np.meshgrid(x, y)

    # Radar path (y=0; fly along x)
    radar_x_positions = np.linspace(-0.8 * space_extent, 0.8 * space_extent, num_positions)
    radar_y = 0.0

    # Targets (one in mainlobe, one in sidelobe azimuth)
    target1_x, target1_y = 0.0,   200.0
    target2_x, target2_y = -100.0, 210.0
    target_amplitude1 = 0.3
    target_amplitude2 = 0.3

    # -----------------------
    # Range-time sampling (for right-hand "echo lines")
    # -----------------------
    sampling_frequency = 10.0 / pulse_duration
    num_range_bins = max(2, int(pulse_time * sampling_frequency))
    t = np.linspace(0, pulse_time, num_range_bins, endpoint=False)
    range_axis = c * t / 2.0

    # 2D echo image (rows = radar positions, cols = range bins)
    echo_data = np.zeros((num_positions, num_range_bins), dtype=float)

    # -----------------------
    # Antenna & pulse helpers
    # -----------------------
    def antenna_pattern(theta_rad):
        """sinc^2 antenna pattern; theta normalized by beamwidth (radians)."""
        bw = np.radians(beamwidth_degrees)
        return np.sinc(theta_rad / bw) ** 2

    def rect(window_t, width=1.0, center=0.0):
        half = width / 2.0
        return ((window_t >= center - half) & (window_t <= center + half)).astype(float)

    # -----------------------
    # Widgets (Voilá-friendly)
    # -----------------------
    play = Play(value=0, min=0, max=total_frames, step=1, interval=120)
    f_slider = IntSlider(value=0, min=0, max=total_frames, step=1, description="Frame")
    jslink((play, 'value'), (f_slider, 'value'))
    readout = Label()
    controls = HBox([play, f_slider], layout=Layout(align_items='center', column_gap='12px'))

    # -----------------------
    # Figure (left = wave map, right = echo image)
    # -----------------------
    fig = plt.figure(figsize=(12, 6))
    gs = fig.add_gridspec(1, 2, width_ratios=[1.05, 1.0], wspace=0.28)
    ax_wave = fig.add_subplot(gs[0, 0])
    ax_sar  = fig.add_subplot(gs[0, 1])

    extent = (-space_extent, space_extent, 0, 2 * space_extent)  # origin="lower"
    im_wave = ax_wave.imshow(np.zeros_like(X), extent=extent, origin="lower",
                             cmap='jet', vmin=-25, vmax=0)
    fig.colorbar(im_wave, ax=ax_wave, fraction=0.046, pad=0.04, label='Field (dB)')
    ax_wave.set_title("SAR Data Collection (Single Outgoing Pulse + Reflections)")
    ax_wave.set_xlabel("X Position (m)")
    ax_wave.set_ylabel("Y Position (m)")

    # Targets and radar/aperture overlays
    ax_wave.scatter(target1_x, target1_y, color="red",  s=100, marker="x", label="Target in mainlobe")
    ax_wave.scatter(target2_x, target2_y, color="cyan", s=100, marker="x", label="Target in sidelobe")
    sc_radar = ax_wave.scatter([], [], color="white", s=50, marker='x', label="Radar")
    line_aperture, = ax_wave.plot([], [], color='white', linewidth=2, label="Synthetic Aperture")
    ax_wave.legend(loc="upper left")

    # Right: echo heatmap (range vs position)
    im_echo = ax_sar.imshow(10*np.log10(echo_data + 1e-12), aspect='auto', cmap='jet',
                            extent=[range_axis[0], range_axis[-1], num_positions, 0])
    cb = fig.colorbar(im_echo, ax=ax_sar, fraction=0.046, pad=0.04, label='Echo (dB)')
    ax_sar.set_title("Accumulated SAR Echo (Range vs. Aperture Step)")
    ax_sar.set_xlabel("Range (m)")
    ax_sar.set_ylabel("Radar Position Index")

    # Prevent static snapshot in Voilá
    plt.close(fig)

    out = Output()

    # Aperture trace state
    aperture_x, aperture_y = [], []

    # -----------------------
    # Frame updater (course-style redraw)
    # -----------------------
    def _update(frame_idx: int):
        # Map frame -> radar position + subframe
        pos_idx   = frame_idx // wave_frames
        sub_idx   = frame_idx %  wave_frames

        # Clamp (avoid index errors if slider hits end)
        pos_idx = min(pos_idx, num_positions - 1)
        radar_x = radar_x_positions[pos_idx]

        # Grow aperture polyline at the start of each new position
        if sub_idx == 0 and (len(aperture_x) == 0 or aperture_x[-1] != radar_x):
            aperture_x.append(radar_x)
            aperture_y.append(radar_y)
            line_aperture.set_data(aperture_x, aperture_y)

        # Geometry to targets
        dist1 = np.hypot(radar_x - target1_x, radar_y - target1_y)
        dist2 = np.hypot(radar_x - target2_x, radar_y - target2_y)

        # Azimuth angles to targets
        az1 = np.arctan2(target1_x - radar_x, target1_y - radar_y)
        az2 = np.arctan2(target2_x - radar_x, target2_y - radar_y)

        # Fixed boresight: point to middle of the scene (x=0, y=space_extent)
        az_bore = np.arctan2(0.0 - radar_x, space_extent - radar_y)

        # Current time within the pulse window
        current_time = (sub_idx / (wave_frames - 1)) * pulse_time if wave_frames > 1 else 0.0

        # Distances from radar to all pixels & grid angles
        R_arr = np.hypot(X - radar_x, Y - radar_y)
        theta_grid = np.arctan2(X - radar_x, Y - radar_y)

        # ---------- Single outgoing pulse (once!) ----------
        pulse_distance = current_time * c
        pulse_width_m  = pulse_duration * c
        outgoing_mask = (R_arr > (pulse_distance - pulse_width_m/2)) & (R_arr < (pulse_distance + pulse_width_m/2))
        # Weight outgoing by antenna pattern relative to boresight
        pat_grid = antenna_pattern(theta_grid - az_bore)
        outgoing = outgoing_mask.astype(float) * pat_grid

        # ---------- Reflections (two targets) ----------
        # Transmit gain toward each target (no second outgoing pulse!)
        tx_gain_1 = antenna_pattern(az1 - az_bore)
        tx_gain_2 = antenna_pattern(az2 - az_bore)

        # Rings centered at targets when the pulse has reached them and come back "leftover"
        Z_reflect = np.zeros_like(R_arr)
        if pulse_distance >= dist1:
            leftover1 = pulse_distance - dist1
            R_t1 = np.hypot(X - target1_x, Y - target1_y)
            m_ref1 = (R_t1 > (leftover1 - pulse_width_m/2)) & (R_t1 < (leftover1 + pulse_width_m/2))
            Z_reflect[m_ref1] += target_amplitude1 * tx_gain_1

        if pulse_distance >= dist2:
            leftover2 = pulse_distance - dist2
            R_t2 = np.hypot(X - target2_x, Y - target2_y)
            m_ref2 = (R_t2 > (leftover2 - pulse_width_m/2)) & (R_t2 < (leftover2 + pulse_width_m/2))
            Z_reflect[m_ref2] += target_amplitude2 * tx_gain_2

        Z = outgoing + Z_reflect

        # Left panel update
        im_wave.set_data(10.0 * np.log10(Z + 1e-12))
        sc_radar.set_offsets([radar_x, radar_y])

        # After final subframe at this position, write one range line (right panel)
        if sub_idx == (wave_frames - 1):
            rt1 = 2.0 * dist1 / c
            rt2 = 2.0 * dist2 / c
            # Echo amplitudes include transmit pattern toward each target
            echo1 = (target_amplitude1 * tx_gain_1) * rect(t, pulse_duration, rt1)
            echo2 = (target_amplitude2 * tx_gain_2) * rect(t, pulse_duration, rt2)
            echo_data[pos_idx, :] = echo1 + echo2

            im_echo.set_data(10.0 * np.log10(echo_data + 1e-12))
            max_val = 10.0 * np.log10(
                max(target_amplitude1, target_amplitude2) + 1e-12
            )
            im_echo.set_clim(max_val - 25.0, max_val)

        # UI readout
        readout.value = (
            f"Frame {frame_idx}/{total_frames}  |  pos {pos_idx+1}/{num_positions}  |  "
        )

        # Redraw both panes (Voilá-friendly)
        with out:
            out.clear_output(wait=True)
            display(fig)

    # Wire up
    f_slider.observe(lambda ch: _update(ch["new"]), names='value')

    # Initial draw
    _update(f_slider.value)

    return VBox([controls, readout, out])

az_ambiguity_demo = build_azimuth_ambiguities_demo()
display(az_ambiguity_demo)


## 👻 From Echoes to Ghosts: How Azimuth Ambiguities Appear in the Image

We just looked at how sidelobe targets sneak their echoes into the raw data. Now let’s take the next step: **what does the actual SAR image look like** in this situation?

---

### 🛠️ How SAR Beamforming Normally Works

When building a SAR image, the processor does two essential things for every pixel:

1. **Trace the correct range curve** 📈  
   For each pixel location on the ground, it figures out the exact distance the radar signal should have traveled at every aperture position.

2. **Apply phase corrections** 🔄  
   Those distances translate into phase shifts. By lining them up correctly, the returns from that pixel add **coherently**, creating a sharp focused spot.

That’s how a single bright target in the main beam turns into a clear, focused point in the SAR image.

---

### 🔀 What Changes with a Sidelobe Target

Here’s the twist: if a target sits in a **sidelobe**, its echoes still leak into the data.

- At some pixel locations, the sidelobe echoes will **add coherently**, just like grating lobes in a conventional antenna array.  
- But the **range history** of that sidelobe target is different from the mainlobe target. It doesn’t match the pixel grid the processor is expecting. ⚡

---

### 🔦 The Result: A Ghost in the Image

Because the sidelobe target’s echoes don’t line up with any “real” pixel:

- Its energy doesn’t focus into a single sharp point.  
- Instead, it gets **smeared out across a wide area**.  
- To the analyst, it shows up as a **fuzzy, misplaced “ghost” signature** 👻 lurking in the image.

---

Even though a sidelobe target isn’t in the main beam, its echoes can still **contaminate the final SAR image** with distracting ghost features. That’s why azimuth ambiguities are not just a theoretical curiosity — they’re a **real image quality challenge** that radar engineers must tackle.  

Mitigation requires careful system design, including:  
- 🛰️ **Antennas with low sidelobe levels** to reduce leakage.  
- 🎚️ **Well-chosen PRF values** to balance azimuth sampling against unambiguous range.  

---

